In [56]:
import requests
import csv
from datetime import datetime, timedelta

def fetch_weather_data(latitude, longitude, start_date, end_date):
    # link can be changed for different locations, data, etc
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude=37.7749&longitude=-122.4194&start_date=2024-06-14&end_date=2024-06-28&hourly=temperature_2m"
    response = requests.get(url)
    return response.json()

def process_weather_data(data):
    if 'error' in data:
        print(f"API Error: {data['reason']}")
        return None

    if 'hourly' not in data:
        print("Error: 'hourly' key not found in the API response.")
        return None
    
    temperatures = data['hourly']['temperature_2m']
    
    print("\nTemperatures (first 10 entries):")
    for temp in temperatures[:10]:
        print(f"{temp}°C")
    print("...")
    
    return temperatures

def write_to_csv(data, latitude, longitude, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Longitude', 'Latitude', 'Time', 'Temperature'])
        for row in data:
            writer.writerow([longitude, latitude, row[0], row[1]])  # Fixed this line

def main():
    latitude = 52.52
    longitude = 13.41
    
    # Use 7 days ago as start date and yesterday as end date
    end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=8)).strftime("%Y-%m-%d")
    
    print(f"Fetching weather data for coordinates ({latitude}, {longitude}) from {start_date} to {end_date}")
    
    weather_data = fetch_weather_data(latitude, longitude, start_date, end_date)
    temperatures = process_weather_data(weather_data)
    
    if not temperatures:
        print("Unable to process weather data. Exiting.")
        return
    
    csv_data = list(zip(weather_data['hourly']['time'], temperatures))
    
    write_to_csv(csv_data, latitude, longitude, 'weather_data.csv')
    print("\nData has been written to weather_data.csv")

if __name__ == "__main__":
    main()

Fetching weather data for coordinates (52.52, 13.41) from 2024-06-22 to 2024-06-29

Temperatures (first 10 entries):
15.5°C
13.9°C
13.6°C
13.0°C
12.7°C
12.6°C
12.4°C
12.1°C
12.1°C
12.2°C
...

Data has been written to weather_data.csv


---
## testing below
---

In [44]:
# Check if Temperature is above 95º and humidity is below 30%, if so fire risk is True and viceversa
# Then place location and fire risk boolean value into a csv file.

import requests
import csv

# Latitude and Longitude for San Francisco, CA
latitude = 37.77056
longitude = -122.42694

# Construct the endpoint URL
endpoint = f'https://api.weather.gov/points/{latitude},{longitude}'

# Define headers
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Step 1: Make the API request to get grid points
    response = requests.get(endpoint, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the observation stations URL
        observation_stations_url = data['properties']['observationStations']

        # Step 2: Make the API request to get observation stations
        response_stations = requests.get(observation_stations_url, headers=headers)

        if response_stations.status_code == 200:
            stations_data = response_stations.json()
            if 'features' in stations_data and len(stations_data['features']) > 0:
                station_id = stations_data['features'][0]['properties']['stationIdentifier']
                print(f"Station ID: {station_id}")

                # Step 3: Construct the URL to get the latest observation for the station
                latest_observation_url = f'https://api.weather.gov/stations/{station_id}/observations/latest'

                # Step 4: Make the API request to get the latest observation
                response_observation = requests.get(latest_observation_url, headers=headers)

                if response_observation.status_code == 200:
                    observation_data = response_observation.json()

                    # Extract relevant weather information
                    properties = observation_data['properties']
                    temperature_c = properties['temperature']['value']  # Temperature in Celsius
                    temperature_f = (temperature_c * 9/5) + 32  # Temperature in Fahrenheit
                    humidity = properties['relativeHumidity']['value']  # Relative humidity in %

                    # Check conditions for fire risk
                    above_90 = temperature_f > 90
                    fire_risk = above_90 and humidity < 30

                    # Print weather details
                    print(f"Weather in San Francisco:")
                    print(f"Temperature: {temperature_c:.2f}°C / {temperature_f:.2f}°F")
                    print(f"Humidity: {humidity:.2f}%")
                    print(f"Fire Risk: {'True' if fire_risk else 'False'}")

                    # Prepare data to write to CSV
                    weather_data = [
                        ['Location', 'Fire Risk'],
                        ['San Francisco, CA', fire_risk]
                    ]

                    # Write data to CSV file
                    with open('weather_alerts.csv', mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerows(weather_data)

                    print("Weather alert data written to weather_alerts.csv")

                else:
                    print(f"Error fetching latest observation: {response_observation.status_code}")
                    print(response_observation.json())
            else:
                print("No observation stations found.")
        else:
            print(f"Error fetching observation stations: {response_stations.status_code}")
            print(response_stations.json())
    else:
        print(f"Error fetching grid points: {response.status_code}")
        print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Station ID: SFOC1
Weather in San Francisco:
Temperature: 18.09°C / 64.56°F
Humidity: 61.98%
Fire Risk: False
Weather alert data written to weather_alerts.csv


In [40]:
import requests
import pandas as pd

url = "https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2024-06-14&end_date=2024-06-28&hourly=temperature_2m,relative_humidity_2m"

try:
    response = requests.get(url)
    data = response.json()

    # Check if data contains the expected keys
    if "hourly" in data:
        hourly_data = data["hourly"]

        # Initialize lists to store data
        dates = []
        temperatures = []
        humidities = []

        # Check if hourly_data is a list of dictionaries
        if isinstance(hourly_data, list) and all(isinstance(item, dict) for item in hourly_data):
            # Extract date, temperature, and humidity from each hourly data point
            for hour_data in hourly_data:
                dates.append(pd.to_datetime(hour_data.get("time", None), unit="s", utc=True))
                temperatures.append(hour_data.get("temperature_2m", None))
                humidities.append(hour_data.get("relative_humidity_2m", None))

            # Create DataFrame
            hourly_dataframe = pd.DataFrame({
                "Date": dates,
                "Temperature (°C)": temperatures,
                "Humidity (%)": humidities
            })

            # Print neatly formatted output
            print(hourly_dataframe)

        else:
            print("Unexpected structure in hourly_data:", hourly_data)

    else:
        print("No hourly data found in the API response.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except KeyError as e:
    print(f"KeyError: {str(e)}. Check the structure of the API response and adjust your code accordingly.")


Unexpected structure in hourly_data: {'time': ['2024-06-14T00:00', '2024-06-14T01:00', '2024-06-14T02:00', '2024-06-14T03:00', '2024-06-14T04:00', '2024-06-14T05:00', '2024-06-14T06:00', '2024-06-14T07:00', '2024-06-14T08:00', '2024-06-14T09:00', '2024-06-14T10:00', '2024-06-14T11:00', '2024-06-14T12:00', '2024-06-14T13:00', '2024-06-14T14:00', '2024-06-14T15:00', '2024-06-14T16:00', '2024-06-14T17:00', '2024-06-14T18:00', '2024-06-14T19:00', '2024-06-14T20:00', '2024-06-14T21:00', '2024-06-14T22:00', '2024-06-14T23:00', '2024-06-15T00:00', '2024-06-15T01:00', '2024-06-15T02:00', '2024-06-15T03:00', '2024-06-15T04:00', '2024-06-15T05:00', '2024-06-15T06:00', '2024-06-15T07:00', '2024-06-15T08:00', '2024-06-15T09:00', '2024-06-15T10:00', '2024-06-15T11:00', '2024-06-15T12:00', '2024-06-15T13:00', '2024-06-15T14:00', '2024-06-15T15:00', '2024-06-15T16:00', '2024-06-15T17:00', '2024-06-15T18:00', '2024-06-15T19:00', '2024-06-15T20:00', '2024-06-15T21:00', '2024-06-15T22:00', '2024-06-15T2

In [47]:
# cleaned up version

import requests
import csv
from datetime import datetime, timedelta

def fetch_weather_data(latitude, longitude, start_date, end_date):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relative_humidity_2m"
    response = requests.get(url)
    return response.json()

def process_weather_data(data):
    if 'error' in data:
        print(f"API Error: {data['reason']}")
        return None, None

    if 'hourly' not in data:
        print("Error: 'hourly' key not found in the API response.")
        return None, None
    
    temperatures = data['hourly']['temperature_2m']
    humidities = data['hourly']['relative_humidity_2m']
    
    print("\nTemperatures (first 10 entries):")
    for temp in temperatures[:10]:
        print(f"{temp}°C")
    print("...")

    print("\nHumidities (first 10 entries):")
    for humidity in humidities[:10]:
        print(f"{humidity}%")
    print("...")
    
    return temperatures, humidities

def calculate_fire_risk(temperature, humidity):
    return temperature > 40 and humidity < 30

def write_to_csv(data, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'Temperature', 'Humidity', 'Fire Risk'])
        writer.writerows(data)

def main():
    latitude = 52.52
    longitude = 13.41
    
    # Use 7 days ago as start date and yesterday as end date
    end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=8)).strftime("%Y-%m-%d")
    
    print(f"Fetching weather data for coordinates ({latitude}, {longitude}) from {start_date} to {end_date}")
    
    weather_data = fetch_weather_data(latitude, longitude, start_date, end_date)
    temperatures, humidities = process_weather_data(weather_data)
    
    if not temperatures or not humidities:
        print("Unable to process weather data. Exiting.")
        return
    
    csv_data = []
    for date, temp, humidity in zip(weather_data['hourly']['time'], temperatures, humidities):
        fire_risk = calculate_fire_risk(temp, humidity)
        csv_data.append([date, temp, humidity, fire_risk])
    
    write_to_csv(csv_data, 'weather_fire_risk.csv')
    print("\nData has been written to weather_fire_risk.csv")

if __name__ == "__main__":
    main()

Fetching weather data for coordinates (52.52, 13.41) from 2024-06-21 to 2024-06-28

Temperatures (first 10 entries):
16.9°C
15.6°C
15.3°C
15.3°C
16.0°C
17.1°C
18.8°C
19.3°C
21.1°C
23.4°C
...

Humidities (first 10 entries):
66%
78%
78%
78%
78%
77%
73%
74%
70%
65%
...

Data has been written to weather_fire_risk.csv


In [ ]:
# in csv have long, lat, time, temp, other data maybe

In [49]:
import requests
import csv
from datetime import datetime, timedelta

def fetch_weather_data(latitude, longitude, start_date, end_date):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly=temperature_2m"
    response = requests.get(url)
    return response.json()

def process_weather_data(data):
    if 'error' in data:
        print(f"API Error: {data['reason']}")
        return None

    if 'hourly' not in data:
        print("Error: 'hourly' key not found in the API response.")
        return None
    
    temperatures = data['hourly']['temperature_2m']
    
    print("\nTemperatures (first 10 entries):")
    for temp in temperatures[:10]:
        print(f"{temp}°C")
    print("...")
    
    return temperatures

def write_to_csv(data, latitude, longitude, filename):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Longitude', 'Latitude', 'Time', 'Temperature'])
        for row in data:
            writer.writerow([longitude, latitude, row[0], row[1]])  # Fixed this line

def main():
    latitude = 52.52
    longitude = 13.41
    
    # Use 7 days ago as start date and yesterday as end date
    end_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=8)).strftime("%Y-%m-%d")
    
    print(f"Fetching weather data for coordinates ({latitude}, {longitude}) from {start_date} to {end_date}")
    
    weather_data = fetch_weather_data(latitude, longitude, start_date, end_date)
    temperatures = process_weather_data(weather_data)
    
    if not temperatures:
        print("Unable to process weather data. Exiting.")
        return
    
    csv_data = list(zip(weather_data['hourly']['time'], temperatures))
    
    write_to_csv(csv_data, latitude, longitude, 'weather_data.csv')
    print("\nData has been written to weather_data.csv")

if __name__ == "__main__":
    main()

Fetching weather data for coordinates (52.52, 13.41) from 2024-06-21 to 2024-06-28

Temperatures (first 10 entries):
16.9°C
15.6°C
15.3°C
15.3°C
16.0°C
17.1°C
18.8°C
19.3°C
21.1°C
23.4°C
...

Data has been written to weather_data.csv


In [14]:
import requests

def fetch_noaa_data(station_id, dataset_id, start_date, end_date):
    base_url = 'https://www.ncei.noaa.gov/access/services/data/v1'
    endpoint = f'{base_url}/data'
    
    params = {
        'dataset': dataset_id,
        'stations': station_id,
        'startDate': start_date,
        'endDate': end_date,
        'dataTypes': 'TMIN,TMAX',
        'includeAttributes': 'false',
        'format': 'json',
        'units': 'standard',
    }
    
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching NOAA data: {e}")
        return None

# Example usage
if __name__ == "__main__":
    station_id = 'GHCND:USW00023272'  # Example station ID for San Francisco
    dataset_id = 'GSOM'  # Global Summary of the Month dataset
    start_date = '2023-01-01'
    end_date = '2023-12-31'
    
    data = fetch_noaa_data(station_id, dataset_id, start_date, end_date)
    
    if data:
        print(f"Weather data fetched successfully:")
        for result in data['results']:
            date = result['date']
            tmin = result['value']['TMIN']
            tmax = result['value']['TMAX']
            print(f"Date: {date}, TMIN: {tmin}, TMAX: {tmax}")
    else:
        print("Failed to fetch NOAA data.")


Error fetching NOAA data: 404 Client Error:  for url: https://www.ncei.noaa.gov/access/services/data/v1/data?dataset=GSOM&stations=GHCND%3AUSW00023272&startDate=2023-01-01&endDate=2023-12-31&dataTypes=TMIN%2CTMAX&includeAttributes=false&format=json&units=standard
Failed to fetch NOAA data.


In [32]:
import requests

# Latitude and Longitude for San Francisco, CA
latitude = 37.77056
longitude = -122.42694

# Construct the endpoint URL
endpoint = f'https://api.weather.gov/points/{latitude},{longitude}'

# Define headers
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

# Step 1: Make the API request to get grid points
response = requests.get(endpoint, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Extract the observation stations URL
    observation_stations_url = data['properties']['observationStations']
    
    # Step 2: Make the API request to get observation stations
    response_stations = requests.get(observation_stations_url, headers=headers)
    
    if response_stations.status_code == 200:
        stations_data = response_stations.json()
        if 'features' in stations_data and len(stations_data['features']) > 0:
            station_id = stations_data['features'][0]['properties']['stationIdentifier']
            print(f"Station ID: {station_id}")
            
            # Step 3: Construct the URL to get the latest observation for the station
            latest_observation_url = f'https://api.weather.gov/stations/{station_id}/observations/latest'
            
            # Step 4: Make the API request to get the latest observation
            response_observation = requests.get(latest_observation_url, headers=headers)
            
            if response_observation.status_code == 200:
                observation_data = response_observation.json()
                
                # Extract relevant weather information
                properties = observation_data['properties']
                temperature = properties['temperature']['value']  # Temperature in Celsius
                wind_speed = properties['windSpeed']['value']  # Wind speed in m/s
                humidity = properties['relativeHumidity']['value']  # Relative humidity in %
                weather = properties['textDescription']  # Weather description

                # Convert temperature to Fahrenheit if it's not None
                temperature_f = (temperature * 9/5) + 32 if temperature is not None else None

                # Print the weather information with checks for None values
                print(f"Weather in San Francisco:")
                if temperature is not None and temperature_f is not None:
                    print("High fire risk: temperatures above 90°F (32°C) with dry conditions.")
                    print(f"Temperature: {temperature:.2f}°C / {temperature_f:.2f}°F")
                else:
                    print("Temperature: Not available")

                if wind_speed is not None:
                    print(f"Wind Speed: {wind_speed:.2f} m/s")
                else:
                    print("Wind Speed: Not available")

                if humidity is not None:
                    print(f"Humidity: {humidity:.2f}%")
                else:
                    print("Humidity: Not available")

                if weather is not None:
                    print(f"Conditions: {weather}")
                else:
                    print("Conditions: Not available")
            else:
                print(f"Error fetching latest observation: {response_observation.status_code}")
                print(response_observation.json())
        else:
            print("No observation stations found.")
    else:
        print(f"Error fetching observation stations: {response_stations.status_code}")
        print(response_stations.json())
else:
    print(f"Error fetching grid points: {response.status_code}")
    print(response.json())


Station ID: SFOC1
Weather in San Francisco:
High fire risk: temperatures above 90°F (32°C) with dry conditions.
Temperature: 18.09°C / 64.56°F
Wind Speed: Not available
Humidity: 61.98%
Conditions: 


In [16]:
import requests

# Define the NOAA Alerts API endpoint
alerts_endpoint = 'https://api.weather.gov/alerts'

# Define headers for the request
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Make the API request to fetch alerts
    response = requests.get(alerts_endpoint, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if there are any alerts
        if 'features' in data and len(data['features']) > 0:
            print("Alerts:")
            for alert in data['features']:
                properties = alert['properties']
                event = properties['event']
                headline = properties['headline']
                severity = properties['severity']
                print(f"- Event: {event}")
                print(f"  Headline: {headline}")
                print(f"  Severity: {severity}")
                print()
        else:
            print("No alerts found.")

    else:
        print(f"Error fetching alerts: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Alerts:
- Event: Special Weather Statement
  Headline: Special Weather Statement issued June 29 at 6:24PM CDT by NWS Amarillo TX
  Severity: Moderate

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 6:24PM CDT until June 29 at 7:15PM CDT by NWS Tulsa OK
  Severity: Severe

- Event: Special Weather Statement
  Headline: Special Weather Statement issued June 29 at 7:23PM EDT by NWS Gaylord MI
  Severity: Moderate

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 7:23PM EDT until June 29 at 7:30PM EDT by NWS Wilmington OH
  Severity: Severe

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 7:22PM EDT until June 29 at 7:45PM EDT by NWS Charleston WV
  Severity: Severe

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 6:22PM CDT until June 29 at 6:45PM CDT by NWS Norman OK
  Severity: Severe

- Event: Severe 

In [24]:
# Same code as above, but filtered for San Francisco CA
import requests

# Define the NOAA Alerts API endpoint
alerts_endpoint = 'https://api.weather.gov/alerts'

# Define headers for the request
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Make the API request to fetch alerts
    response = requests.get(alerts_endpoint, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if there are any alerts
        if 'features' in data and len(data['features']) > 0:
            print("Alerts for San Francisco, CA:")
            for alert in data['features']:
                properties = alert['properties']
                event = properties['event']
                headline = properties['headline']
                severity = properties['severity']
                
                # Ensure headline is not None and ends with "San Francisco CA"
                if headline and isinstance(headline, str) and headline.endswith("San Francisco CA"):
                    print(f"- Event: {event}")
                    print(f"  Headline: {headline}")
                    print(f"  Severity: {severity}")
                    print()
        
        else:
            print("No alerts found.")

    else:
        print(f"Error fetching alerts: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Alerts for San Francisco, CA:
- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until June 30 at 3:00PM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 3:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until June 30 at 9:00PM PDT by NWS San Francisco CA
  Severity: Min